# Business Analytics

This is an example of my code that I completed in homework #3 in the business analytics course.

In [1]:
import scipy.stats as ss
import numpy as np
import cvxpy as cp

# IMPORTANT: in this assignment, you DO NOT need to simulate anything. Use the exact distributions given to you.

- **Due Date:** 2022-10-15 23:59:59
- **Late Submission:** not allowed
- **Total Points:** 15 points (each question is worth 1 point)
- **Submission Instructions:**
    - Submit your assignment as a Jupyter notebook (`.ipynb`) file on my.nes
    - Make sure to save your answers in the corresponding variables (e. g. ans_1A for problem 1, question A)
    - Make sure to check that your notebook runs without errors and finishes in a reasonable amount of time (1 hour in Colab notebook)

In [7]:
print("ans_1A", ans_1A)
print("ans_1B", ans_1B)
print("ans_1C", ans_1C)

print("ans_2A", ans_2A)
print("ans_2B", ans_2B)
print("ans_2C", ans_2C)
print("ans_2D", ans_2D)
print("ans_2E", ans_2E)
print("ans_2F", ans_2F)

print("ans_3A", ans_3A)
print("ans_3B", ans_3B)
print("ans_3C", ans_3C)

print("ans_4A", ans_4A)
print("ans_4B", ans_4B)
print("ans_4C", ans_4C)

ans_1A 20
ans_1B 2171.4285716581876
ans_1C 34.54545454405743
ans_2A [1.11441842 0.96452969 0.6657495  1.13697285]
ans_2B 0.4746596753491127
ans_2C 99.012
ans_2D 95.934
ans_2E 113.886
ans_2F 117.9
ans_3A 800.0000002678655
ans_3B 564.0718776793368
ans_3C None
ans_4A {'Blue': 2676.999999824616, 'Red': 1193.9999999813324, 'Green': 2253.0000000145305, 'Purple': 1127.9999999857234}
ans_4B 3396.1498517323093
ans_4C {'Undyed': 1.3047083527325273e-05, 'Blue': 2676.9999957160894, 'Red': 1193.999984882366, 'Green': 2252.9999986848084, 'Purple': 1127.9999969417634}


### Problem 1 (Newsvendor)

The demand for a product can take any value between 20 and 40 units. The
selling price is assumed to be \$110 and the purchase cost is \$30, and
there is no salvage opportunity for unsold units. The company decides on
its order quantity.

**QUESTIONS**

A. Suppose that the demand distribution is controlled by an adversary.
Whichever order quantity the company chooses, the adversary would pick
up the distribution that would minimize the expected profit under this
order quantity. What is the order quantity that maximizes the profit in
this case? (Notice that the demand must still be between 20 and 40
units).

B. What is the optimal expected profit of the company if all possible
demand values are equally likely?

Now assume that the company is looking to minimize its *maximum regret*, meaning the difference between the best possible payoff that could have been achieved with the benefit of hindsight and the realized payoff. For example, if the actual demand is 25 units, and the ordering decision is 30 units, the company's regret is computed as follows. The best possible payoff is \$2000, being equal to (110 - 30)*25. The realized payoff is 110 * 25 - 30 * 30 = \$1850. The regret, is, therefore, 2000 - 1850 = \$150.  

C. What is the order quantity that minimizes the maximum regret?

In [8]:
# In your code, assign the answers to the following variables:
ans_1A = None # Optimal order quantity in 1A
ans_1B = None # Optimal profit in 1B
ans_1C = None # Optimal order quantity in 1C
#your code goes here

# 1A
# He should choose 20 because his adversary will always choose the minimal demand of 20
# There is no sense to produce more or less
ans_1A = 20

# 1B
r, c = 110, 30
demand = [20 + k for k in range(21)]
probs = [1/21]*21

# order quantity
q = cp.Variable(nonneg = True)

# sales in every demand scenario
z = cp.Variable(21)

# maximize revenue - ordering cost
obj = cp.Maximize(np.sum([probs[k]*(r * z[k] - c * q) for k in range(21)]))
constraints = []

# cannot sell more than either demand or order quantity
for k in range(21):
    constraints.append(z[k] <= q)
    constraints.append(z[k] <= demand[k])
nv_prob = cp.Problem(obj, constraints)
nv_prob.solve()
ans_1B = nv_prob.value

# 1C
r, c = 110, 30
demand = [20 + k for k in range(21)]
probs = [1/21]*21

# order quantity
q = cp.Variable(nonneg = True)

# sales in every demand scenario
z = cp.Variable(21)

obj = cp.Minimize(
    cp.max(cp.vstack(
    [((r - c) * demand[k] - r * z[k] + c * q) for k in range(21)]
    ))
)
constraints = []

# cannot sell more than either demand or order quantity
for k in range(21):
    constraints.append(z[k] <= q)
    constraints.append(z[k] <= demand[k])
nv_prob = cp.Problem(obj, constraints)
nv_prob.solve()
ans_1C = q.value
ans_1C

print("ans_1A:", ans_1A)
print("ans_1B:", ans_1B)
print("ans_1C:", ans_1C)

ans_1A: 20
ans_1B: 2171.4285716581876
ans_1C: 34.54545454405743


### Problem 2 (Flexibility)

A bespoke automotive manufacturer, Carrus, plans to offer four car
models: Prima, Secunda, Tertia and Quadra. Each model would sell for
\$110,000 and have a unit variable cost of \$50,000. Also, Carrus owns
four auto plants, named after their locations: Arlington, Bearington,
Covington, and Dearborn. Equipping any plant to produce any given car
model would cost \$3M in fixed costs. (For example, Carrus will pay $48M if every plant is able to produce every model.) Each plant can produce at most
800 cars annually. The annual demand for the car models is uncertain. The uncertainty can be described by the
following table of equally likely scenarios:

|  **Scenario**|  **Prima**|  **Secunda**|  **Tertia**|  **Quadra**|
|--------------|-----------|-------------|------------|------------|
|        1     |      2501 |        2126 |        164 |         247|
|        2     |       267 |         63  |        1548|         331|
|        3     |       169 |         336 |        814 |         212|
|        4     |       291 |         357 |        1214|         248|
|        5     |      1171 |        1531 |        1213|         252|
|        6     |       49  |         164 |        249 |         487|
|        7     |      2821 |         415 |        436 |         161|
|        8     |       230 |         205 |        193 |         623|
|        9     |       678 |         514 |        377 |        2591|
|        10    |       384 |         883 |        837 |        1035|

Carrus is looking to decide on its flexibility investments that would
optimize its expected profits.

**QUESTIONS**

A. What is the coefficient of variation (e. g. standard deviation divided by the expected value) for each of the four models?

B. What is the coefficient variation of the total demand for all car models?

C. If Carrus adopts a fully dedicated strategy (meaning that each plant
can only produce one car model), what would be Carrus' expected profit?

D. If Carrus makes all plants flexible (meaning that each plant can
produce all models), what would be Carrus' expected profit?

E. Carrus considers the following *chaining* policy. Arlington can
produce Prima and Secunda, Bearington can produce Secunda and Tertia,
Covington can produce Tertia and Quadra, and Dearborn can produce Quadra
and Prima. What is the expected profit under this policy?

F. What is the optimal flexibility decision for Carrus? What is the optimal expected profit?

In [3]:
scenarios = np.array([
[2501, 2126, 164, 247],
[267, 63, 1548, 331],
[169, 336, 814, 212],
[291, 357, 1214, 248],
[1171, 1531, 1213, 252],
[49, 164, 249, 487],
[2821, 415, 436, 161],
[230, 205, 193, 623],
[678, 514, 377, 2591],
[384, 883, 837, 1035]
])

# In your code, assign the answers to the following variables:
# standard devations in parts 2A and 2B
# optimal profits in parts 2C to 2F (in million dollars)
ans_2A = None
ans_2B = None
ans_2C = None
ans_2D = None
ans_2E = None
ans_2F = None

# your code goes here
# 2A
ans_2A = np.std(scenarios, axis=0) / np.mean(scenarios, axis=0)

# 2B
ans_2B = np.sum(scenarios, axis=1).std() / np.sum(scenarios, axis=1).mean()

# 2C
r, c = 110000, 50000
inv_cost = 3000000
probs = [1/10]*10

# investments (factory x model)
inv = cp.Variable((4, 4), boolean = True)

# order quantity (factory x model)
q = cp.Variable((4, 4), nonneg = True)

# model sales in every demand scenario (scenario x factory x model)
m_sales = {(k, i, m): cp.Variable(nonneg=True) for k in range(10) for i in range(4) for m in range(4)}

# maximize revenue - ordering cost
profit_one_factory = lambda i: cp.sum([probs[k]*cp.sum([(r - c) * m_sales[(k, i, m)] for m in range(4)]) for k in range(10)])
profit_all_factories = cp.sum([profit_one_factory(i) - cp.sum(inv[i, :]) * inv_cost for i in range(4)])
obj = cp.Maximize(profit_all_factories)
constraints = [
    cp.sum(inv, axis=1) == 1
]

# cannot sell more than either demand or order quantity
for k in range(10):
  for m in range(4):
    for i in range(4):
      constraints.append(m_sales[(k, i, m)] <= inv[i, m] * 800)
    constraints.append(cp.sum([m_sales[(k, i, m)] for i in range(4)]) <= scenarios[k, m])

nv_prob = cp.Problem(obj, constraints)
nv_prob.solve()
ans_2C = nv_prob.value / 1e6

# 2D
r, c = 110000, 50000
inv_cost = 3000000
probs = [1/10]*10

# investments (factory x model)
inv = cp.Variable((4, 4), boolean = True)

# order quantity (factory x model)
q = cp.Variable((4, 4), nonneg = True)

# model sales in every demand scenario (scenario x factory x model)
m_sales = {(k, i, m): cp.Variable(nonneg=True) for k in range(10) for i in range(4) for m in range(4)}

# maximize revenue - ordering cost
profit_one_factory = lambda i: cp.sum([probs[k]*cp.sum([(r - c) * m_sales[(k, i, m)] for m in range(4)]) for k in range(10)])
profit_all_factories = cp.sum([profit_one_factory(i) - cp.sum(inv[i, :]) * inv_cost for i in range(4)])
obj = cp.Maximize(profit_all_factories)
constraints = [
    inv[i, k] == 1 for i in range(4) for k in range(4)
]

# cannot sell more than either demand or order quantity
for k in range(10):
  for m in range(4):
    for i in range(4):
      constraints.append(m_sales[(k, i, m)] <= inv[i, m] * 800)
    constraints.append(cp.sum([m_sales[(k, i, m)] for i in range(4)]) <= scenarios[k, m])

for k in range(10):
  for i in range(4):
    constraints.append(cp.sum([m_sales[(k, i, m)] for m in range(4)]) <= 800)

nv_prob = cp.Problem(obj, constraints)
nv_prob.solve()
ans_2D = nv_prob.value / 1e6

# 2E
r, c = 110000, 50000
inv_cost = 3000000
probs = [1/10]*10

# investments (factory x model)
inv = cp.Variable((4, 4), boolean = True)

# order quantity (factory x model)
q = cp.Variable((4, 4), nonneg = True)

# model sales in every demand scenario (scenario x factory x model)
m_sales = {(k, i, m): cp.Variable(nonneg=True) for k in range(10) for i in range(4) for m in range(4)}

# maximize revenue - ordering cost
profit_one_factory = lambda i: cp.sum([probs[k]*cp.sum([(r - c) * m_sales[(k, i, m)] for m in range(4)]) for k in range(10)])
profit_all_factories = cp.sum([profit_one_factory(i) - cp.sum(inv[i, :]) * inv_cost for i in range(4)])
obj = cp.Maximize(profit_all_factories)
constraints = [
    inv[i, (i+k)%4] == 1 for i in range(4) for k in range(2)
] + [
    cp.sum(inv[i, :]) == 2 for i in range(4)
]

# cannot sell more than either demand or order quantity
for k in range(10):
  for m in range(4):
    for i in range(4):
      constraints.append(m_sales[(k, i, m)] <= inv[i, m] * 800)
    constraints.append(cp.sum([m_sales[(k, i, m)] for i in range(4)]) <= scenarios[k, m])

for k in range(10):
  for i in range(4):
    constraints.append(cp.sum([m_sales[(k, i, m)] for m in range(4)]) <= 800)

nv_prob = cp.Problem(obj, constraints)
nv_prob.solve()
ans_2E = nv_prob.value / 1e6

# 2F
r, c = 110000, 50000
inv_cost = 3000000
probs = [1/10]*10

# investments (factory x model)
inv = cp.Variable((4, 4), boolean = True)

# order quantity (factory x model)
q = cp.Variable((4, 4), nonneg = True)

# model sales in every demand scenario (scenario x factory x model)
m_sales = {(k, i, m): cp.Variable(nonneg=True) for k in range(10) for i in range(4) for m in range(4)}

# maximize revenue - ordering cost
profit_one_factory = lambda i: cp.sum([probs[k]*cp.sum([(r - c) * m_sales[(k, i, m)] for m in range(4)]) for k in range(10)])
profit_all_factories = cp.sum([profit_one_factory(i) - cp.sum(inv[i, :]) * inv_cost for i in range(4)])
obj = cp.Maximize(profit_all_factories)
constraints = []

# cannot sell more than either demand or order quantity
for k in range(10):
  for m in range(4):
    for i in range(4):
      constraints.append(m_sales[(k, i, m)] <= inv[i, m] * 800)
    constraints.append(cp.sum([m_sales[(k, i, m)] for i in range(4)]) <= scenarios[k, m])

for k in range(10):
  for i in range(4):
    constraints.append(cp.sum([m_sales[(k, i, m)] for m in range(4)]) <= 800)

nv_prob = cp.Problem(obj, constraints)
nv_prob.solve()
ans_2F = nv_prob.value / 1e6

print(ans_2A)
print(ans_2B)
print(ans_2C)
print(ans_2D)
print(ans_2E)
print(ans_2F)

[1.11441842 0.96452969 0.6657495  1.13697285]
0.4746596753491127
99.012
95.934
113.886
117.9


### Problem 3 (Quick Response)

Le Chapeau is a fashion retailer that sells a
fashionable hat with highly uncertain demand. Let $\text{B}(n,p)$ be a binomial distribution with
number of trials $n$ and probability of success $p$. In period 1, the
demand uncertainty is given by the random variable equal to
$100\times\text{B}(n,p)$ having parameters $n=10$ and $p=0.4$ (so that
the expected value is 400 and the maximum value is 1,000). In period 2,
the demand is equal to demand in period 1. Ordering in period 1 costs
\$100; ordering in period 2 costs \$170, and the retail selling price is
\$220. The leftover inventory from period 1 can be used to satisfy the demand in period 2. Units not sold by the end of period 2 cannot be salvaged.

**QUESTIONS**

A. If Le Chapeau does not make use of demand updating (meaning that it only orders in the first period), what would its optimal quantity be?

B. Given that Le Chapeau ordered 500 units in period 1 and sold them
all, what is its expected demand in period 2? (*Hint:* in period 1, you
do not observe the demand, but only sales, which is a censored signal of
demand. Use Bayes rule to update your prior on the distribution of
demand.)

C. Which ordering policy of Le Chapeau would maximize its expected profit? What is the value of this optimal profit?

In [4]:
# In your code, assign the answers to the following variables:
ans_3A = None # Optimal order quantity in 3A
ans_3B = None # Expected demand in 3B
ans_3C = None # Optimal profit in 3C

# your code goes here
# 3A
n, p = 10, 0.4 # 100 * B(n, p) for n = 10 and p = 0.4
demand, probs = zip(*[(100 * k, ss.binom(n, p).pmf(k)) for k in range(n + 1)])

r, c_1, c_2 = 220, 100, 170

sales = cp.Variable((11, 2), nonneg=True)
inv_1 = cp.Variable((11, 2), nonneg=True)
q = cp.Variable(1, nonneg=True)

obj = cp.Maximize(np.sum([
    probs[k] * (r*sales[k, 0] + r*sales[k, 1] - c_1 * q) for k in range(len(probs))
]))
constraints = []

# cannot sell more than either demand or order quantity
for k in range(len(probs)):
    constraints.append(inv_1[k, 0] + sales[k, 0] <= q)
    constraints.append(sales[k, 1] <= inv_1[k, 0])
    constraints.append(sales[k, 0] <= demand[k])
    constraints.append(sales[k, 1] <= demand[k])
nv_prob = cp.Problem(obj, constraints)
nv_prob.solve()
ans_3A = q.value[0]
ans_3A

# 3B
# P(A|B) = P(B|A)*P(A)/P(B) -- Bayes Rule
# B - is that in the first period 500 were sold
# A demand in period 1 and 2
# P(B|A) = 1 for A >= 500 and 0 otherwise
# P(B) = sum(P(B|A)P(A)) over all A => P(B) = sum(P(A)) for A >= 500
p_b = sum([probs[i] for i in range(5, 11)])
new_probs = [0]*5 + [probs[i] / p_b for i in range(5, 11)]
ans_3B = sum([demand[i] * new_probs[i] for i in range(5, 11)])

print(ans_3A)
print(ans_3B)
print(ans_3C)

800.0000002678655
564.0718776793368
None


### Problem 4 (Delayed differentiation)

Luciano, a fashion retailer in Italy, considers launching a new series
of colored sweaters, planning to offer four varieties. Each sweater
variety will be sold at a retail price \$180 and can be reliably
salvaged after the end of the season at a discount price of \$60. Their
demand uncertainty forecast for these four varieties is summarized in
the table below\-\--with all scenarios considered equally likely.

| **Scenario** | **Blue** | **Red** | **Green** | **Purple** |
|--------------|----------|---------|-----------|------------|
|        1     |     407  |      665|        232|         116|
|        2     |     203  |      346|       1725|         298|
|        3     |     233  |      157|        823|         578|
|        4     |     2677 |      347|        366|        1183|
|        5     |     518  |      400|        803|         676|
|        6     |     261  |      886|        457|          65|
|        7     |     596  |     1140|       1528|         363|
|        8     |     4025 |     1250|       2253|        1128|
|        9     |     556  |      656|       4383|         620|
|        10    |     913  |     1194|        122|        1056|


Luciano considers a choice between the three operational models
described in the Questions section below.

**QUESTIONS**

A. Assume that Luciano follows an early differentiation model meaning
that all sweaters are colored in China and are ordered well in advance
of the selling season. Each sweater would have a unit purchase cost of
\$80 under this arrangement. Which order quantities would optimize
Luciano's expected profit?

B. Assume that Luciano adopts a fully delayed differentiation model.
Under this model, undyed sweaters are ordered in China for the unit
purchase cost of \$100 and then dyed on demand as customers purchase
them. This dying on demand costs \$20 per sweater. Which order quantity
of the undyed sweaters would optimize Luciano's expected profit?

C. Assume that Luciano adopts a partial variety postponement policy.
Under this model, some sweaters are ordered undyed and some sweaters are
ordered pre-colored. Which order quantities would optimize Luciano's
expected profit?


In [5]:
# In your code, assign the answers to the following variables:
scenarios = np.array(
[[ 407., 665., 232., 116.],
 [ 203., 346.,1725., 298.],
 [ 233., 157., 823., 578.],
 [2677., 347., 366.,1183.],
 [ 518., 400., 803., 676.],
 [ 261., 886., 457.,  65.],
 [ 596.,1140.,1528., 363.],
 [4025.,1250.,2253.,1128.],
 [ 556., 656.,4383., 620.],
 [ 913.,1194., 122.,1056.]]

)
ans_4A = None # example: {"Blue": 100, "Red":123.4, "Green": 34.5, "Purple" : 12.4} -- Order quantities in 4A
ans_4B = None # example: 1239.39 -- Order quantity of the undyed sweaters in 4B
ans_4C = None # example: {"Undyed":430,"Blue": 100, "Red":123.4, "Green": 34.5, "Purple" : 12.4} -- Order quantities in 4C

# your code goes here
# 4A
q = cp.Variable(4, nonneg=True)
s = cp.Variable((10, 4), nonneg=True)
r, c, salvage = 180, 80, 60
inv = cp.Variable((10, 4), nonneg=True)

profit_one_scenario = lambda k: cp.sum(r * s[k, :] - c * q + salvage * inv[k, :])
obj = cp.Maximize(np.sum([0.1*profit_one_scenario(k) for k in range(10)]))
constraints = []

# cannot sell more than either demand or order quantity
for k in range(10):
  for i in range(4):
    constraints.append(inv[k, i] + s[k, i] <= q[i])
    constraints.append(s[k, i] <= scenarios[k, i])
nv_prob = cp.Problem(obj, constraints)
nv_prob.solve()
ans_4A = dict(zip(['Blue', 'Red', 'Green', 'Purple'], q.value))

# 4B
q = cp.Variable(nonneg=True)
sales = cp.Variable(10, nonneg=True)
inv = cp.Variable(10,  nonneg=True)
r, c, salvage, dying = 180, 100, 60, 20

demand = np.sum(scenarios, axis=1)

profit_one_scenario = lambda k: (r-dying) * sales[k] + (q-sales[k]) * (salvage) - c * q
obj = cp.Maximize(np.sum([0.1*profit_one_scenario(k) for k in range(10)]))
constraints = []

# cannot sell more than either demand or order quantity
constraints.append(sales <= q)
constraints.append(sales <= demand)
nv_prob = cp.Problem(obj, constraints)
nv_prob.solve()
ans_4B = q.value

# 4C
q_undyed = cp.Variable(nonneg=True)
q_colored = cp.Variable(4, nonneg=True)
s_colored = cp.Variable((10, 4), nonneg=True)
s_undyed = cp.Variable(10, nonneg=True)
r, c_1, c_2, salvage, dying = 180, 80, 100, 60, 20
inv_colored = cp.Variable((10, 4), nonneg=True)
inv_undyed = cp.Variable(10, nonneg=True)

profit_one_scenario = lambda k: (r-dying) * s_undyed[k] + cp.sum(r * s_colored[k, :] - c_1 * q_colored + salvage * inv_colored[k, :]) + salvage*inv_undyed[k] - c_2 * q_undyed
obj = cp.Maximize(np.sum([0.1*profit_one_scenario(k) for k in range(10)]))
constraints = []
for k in range(10):
  constraints.append(s_undyed[k] <= cp.sum(scenarios[k, :] - s_colored[k, :]))
  constraints.append(s_undyed[k] + inv_undyed[k] <= q_undyed)
  for i in range(4):
    constraints.append(s_colored[k, i] <= scenarios[k, i])
    constraints.append(s_colored[k, i] + inv_colored[k, i] <= q_colored[i])
nv_prob = cp.Problem(obj, constraints)
nv_prob.solve()
ans_4C = dict(zip(['Undyed', 'Blue', 'Red', 'Green', 'Purple'], np.concatenate([[q_undyed.value], q_colored.value])))

print(ans_4A)
print(ans_4B)
print(ans_4C)

{'Blue': 2676.999999824616, 'Red': 1193.9999999813324, 'Green': 2253.0000000145305, 'Purple': 1127.9999999857234}
3396.1498517323093
{'Undyed': 1.3047083527325273e-05, 'Blue': 2676.9999957160894, 'Red': 1193.999984882366, 'Green': 2252.9999986848084, 'Purple': 1127.9999969417634}
